In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
MNIST = input_data.read_data_sets("/tmp/mnist", one_hot=True)
# note that the data contains
# MNIST.train: 55,000 examples
# MNIST.validation: 5,000 examples
# MNIST.test: 10,000 examples

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/mnist/train-images-idx3-ubyte.gz


Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/mnist/train-labels-idx1-ubyte.gz


Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/mnist/t10k-images-idx3-ubyte.gz


Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/mnist/t10k-labels-idx1-ubyte.gz


In [30]:
# NOTE: each image in MNIST has resolution 28*28 = 784 and each turns to
# a vector of 784 dims
D = 784  # vec_dim
C = 10  # num_classes
batch_size = 128
num_epochs = 28
lr = 0.1
H = 1000
reg = 1e-5

In [13]:
# input - fully connected layer - ReLU - fully connected layer - softmax
W1 = tf.Variable(0.01 * tf.random_normal([D, H]), name='weights_1')
b1 = tf.Variable(tf.zeros([1, H], name='bias_1'))
W2 = tf.Variable(0.01 * tf.random_normal([H, C]), name='weights_2')
b2 = tf.Variable(tf.zeros([1, C], name='bias_2'))

In [6]:
# input placeholders
X = tf.placeholder(tf.float32, [batch_size, D], name='X')
Y = tf.placeholder(tf.float32, [batch_size, C], name='Y')

In [28]:
logits1 = tf.matmul(X, W1) + b1
out1 = tf.nn.relu(logits1, name='out_1')
logits2 = tf.matmul(out1, W2) + b2
loss_batch = tf.nn.softmax_cross_entropy_with_logits(logits=logits2, labels=Y)
loss1 = tf.reduce_mean(loss_batch)

loss_reg = reg * (tf.reduce_sum(tf.square(W1)) + tf.reduce_sum(tf.square(W2))) 
loss = tf.add(loss1, loss_reg, name='loss')
summary_loss = tf.summary.scalar('total_loss', loss)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr).minimize(loss)

In [31]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_batches = int(MNIST.train.num_examples / batch_size)
    writer = tf.summary.FileWriter('./mnist_graphs/', sess.graph)
    print("Training...")
    for i in range(num_epochs):
        cur_loss = 0.0  # current loss for this epoch
        for j in range(num_batches):
            X_batch, Y_batch = MNIST.train.next_batch(batch_size)  # type of X_batch and Y_batch are numpy.ndarray
            _, l, summary = sess.run([optimizer, loss, summary_loss], 
                                     feed_dict={X: X_batch, Y: Y_batch})
            writer.add_summary(summary)
            cur_loss += l
        cur_loss /= num_batches
        print("Epoch {0}: loss = {1}".format(i, cur_loss))
    num_batches_test = int(MNIST.test.num_examples / batch_size)
    acc = 0.0
    print("Testing...")
    for i in range(num_batches_test):
        X_batch, Y_batch = MNIST.test.next_batch(batch_size)
        logits_batch = sess.run(logits2, feed_dict={X: X_batch, Y: Y_batch})
        scores = tf.nn.softmax(logits_batch)
        pred_labels = tf.argmax(scores, axis=1)  # note that scores has shape (batch_size, num_classes)
        correct_labels = tf.argmax(Y_batch, axis=1)  # one hot -> idx of value 1 is also max (the rest = 0)
        correct_preds = tf.equal(pred_labels, correct_labels)
        acc += tf.reduce_sum(tf.cast(correct_preds, tf.float32)).eval()
    print("total accuracy on test set = {0}".format(acc / MNIST.test.num_examples))

Training...


Epoch 0: loss = 0.6593416791001122


Epoch 1: loss = 0.30306128480217676


Epoch 2: loss = 0.25122062452050636


Epoch 3: loss = 0.21524972751701907


...

Epoch 24: loss = 0.043662127858518446


Epoch 25: loss = 0.041640907176498904


Epoch 26: loss = 0.03941749897948303


Epoch 27: loss = 0.04002755907492274
Testing...


total accuracy on test set = 0.9782
